Nella cartella *files* sono presenti 8 file:
- 2 file di testo
- 2 file audio
- 4 immagini,

con varie estensioni.

**Step 1**

Ho iniziato creando, in un notebook, uno script Python che iteri in ordine alfabetico sui file della cartella files e, a seconda del tipo (audio, documento, immagine), li sposti nella relativa sottocartella (qui sotto trovi un esempio). Se la sottocartella non esiste, lo script dovrà crearla automaticamente.

Durante il ciclo, lo script stampa le informazioni dei file: nome, tipo e dimensione in byte. Questo è l'output desiderato:

In [2]:
import os
import fnmatch
import shutil
from os import path
import csv
import numpy as np

#metodo che crea la sotto-cartella se non esiste e sposta il file nella sotto-cartella
def move_file_create_folder(a_file, name_folder, type_name):
    file_size = os.path.getsize('files/' + a_file)
    if not os.path.exists('files/' + name_folder):
        os.mkdir('files/' + name_folder)
    shutil.move('files/' + file, 'files/' + name_folder + '/' + file)
    #stampa in console i dettagli del file
    print(os.path.splitext(a_file)[0] + ' type:' + type_name + ' size:' + str(file_size) + 'B')
    row_csv= np.array([os.path.splitext(a_file)[0], type_name, str(file_size)])
    #inserisce i dati del file nel csv
    with open("files/recap.csv","a") as fd:
        writer = csv.writer(fd)
        writer.writerow(row_csv)
        fd.close()


#controlla se esiste il csv recap
if not path.isfile('files/recap.csv'):
    #se non esiste, viene creato e viene inserito la prima riga: l'header
    data_header= ["name", "type", "size(B)"]
    with open("files/recap.csv", "w") as csv_file:
        writer_csv= csv.writer(csv_file)
        writer_csv.writerow(data_header)
        csv_file.close()

#ciclo su i file presenti nella cartella files
files = os.listdir('files/')
for file in files:
    #controlla se è un file o una cartella, se è un file entra nel blocco if
    if path.isfile('files/' + file):
        #if innestati per poter spostare il file nella giusta sotto-cartella in base al suo tipo
        if fnmatch.fnmatch(file, '*.mp3'):
            move_file_create_folder(file, 'audio', 'audio')
        if fnmatch.fnmatch(file, '*.txt') or fnmatch.fnmatch(file, '*.odt'):
            move_file_create_folder(file, 'docs', 'doc')
        if fnmatch.fnmatch(file, '*.png') or fnmatch.fnmatch(file, '*.jpeg') or fnmatch.fnmatch(file, '*.jpg'):
            move_file_create_folder(file, 'images', 'image')


song1 type:audio size:1087849B
ciao type:doc size:12B
song2 type:audio size:764176B
daffodil type:image size:24657B
trump type:image size:10195B
bw type:image size:94926B
pippo type:doc size:8299B
eclipse type:image size:64243B


Oltre a stamparne le informazioni via via che li sposti, si tiene traccia dei file creando un documento *recap.csv* con le stesse informazioni. Di seguito un esempio in questa cartella.

La struttura finale della cartella files dovrà essere:

        - files            
            - audio
                - song1.mp3
                - song2.mp3
            - docs
                - ciao.txt
                - pippo.odt
            - images
                - bw.png
                - daffodil.jpg
                - eclipse.png
                - trump.jpeg    
            - recap.csv
                
---

**Step 2**

Ho inserito lo script creato in un piccolo eseguibile (chiamalo *addfile.py*) dotato di *interfaccia a linea di comando* (CLI).

Lo scopo dell'eseguibile è spostare un *singolo* file (che si trova nella cartella files) nella sottocartella di competenza, aggiornando il recap.

L'interfaccia dell'eseguibile ha come unico argomento (obbligatorio) il nome del file da spostare (comprensivo di formato, es: 'trump.jpeg'). Nel caso in cui il file passato come argomento non esista, l'interfaccia deve comunicarlo all'utente.

---

**Step 3**

Una immagine in scala di grigio ha un solo livello di colore, una RGB ne ha 3, una RGBA 4 (l'ultimo è detto canale *alpha*).

Il modulo *Image* della libreria *PIL* permette di caricare un'immagine, che può essere trasformata in un array NumPy attraverso la funzione *np.array*. A partire da tale array, è possibile capire se l'immagine caricata è in scala di grigio, RGB o RGBA.

é stato aggiunto al notebook dello Step 1 uno script che iteri sulla sottocartella *images* e costruisca una tabella riassuntiva come questa (prodotta con la libreria *tabulate*):

In [6]:
import numpy as np
from PIL import Image
from tabulate import tabulate
import os

#creo l'array inserendo l'header della tabella
dataTable = np.array([["name", "height", "width", "grayscale", "R", "G", "B", "ALPHA"]])
files = os.listdir('files/images/')
#ciclo tutti file presenti nella sotto-cartella images
for file in files:
    #uso PIL per poter utilizzare l'immagine
    image = Image.open("files/images/" + file)
    #trasformo l'immagine in un array
    imageToArray = np.asarray(image)
    #recupero altezza e larghezza
    heightImage = image.height
    widthImage = image.width
    #inizializzo le variabili che memorizzeranno RGB, la scala dei grigi e opacità
    grayScaleImage = 0
    rColorImage = 0
    gColorImage = 0
    bColorImage = 0
    alphaColorImage = 0
    #controllo la dimensione dell'immagine se sono 2 allora ha solo la scala dei grigi
    if imageToArray.ndim == 2:
        grayScale= np.mean(imageToArray)
        grayScaleImage = round(grayScale, 2)
    else:
        #creo una lista di valori medi per ottenere l'RGB
        mean_rgb= np.mean(imageToArray, axis=(0,1))
        #memorizzo i valori RGB
        rColorImage = round(mean_rgb[0], 2)
        gColorImage = round(mean_rgb[1], 2)
        bColorImage = round(mean_rgb[2], 2)
        #se la lunghezza dei valori medi è 4 allora è presente l'opacità che deve essere memorizzata
        if len(mean_rgb) == 4:
            alphaColorImage = round(mean_rgb[3], 2)
    #per ogni immagine creo una riga nell'array dataTable per salvare tutti i dati richiesti
    dataTable = np.append(dataTable, [file, str(heightImage), str(widthImage), str(grayScaleImage) , str(rColorImage), str(gColorImage), str(bColorImage), str(alphaColorImage)])
#faccio un reshape dell'array per poter sfruttare la libreria tabulate
dataTable = np.reshape(dataTable, (len(files) + 1 ,8))
print(tabulate(dataTable, headers="firstrow",tablefmt='fancy_grid'))

╒══════════════╤══════════╤═════════╤═════════════╤════════╤════════╤═══════╤═════════╕
│ name         │   height │   width │   grayscale │      R │      G │     B │   ALPHA │
╞══════════════╪══════════╪═════════╪═════════════╪════════╪════════╪═══════╪═════════╡
│ daffodil.jpg │      500 │     335 │        0    │ 109.23 │  85.52 │  4.77 │    0    │
├──────────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ trump.jpeg   │      183 │     275 │        0    │  97.01 │  98.99 │ 90.92 │    0    │
├──────────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ bw.png       │      512 │     512 │       21.48 │   0    │   0    │  0    │    0    │
├──────────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ eclipse.png  │      256 │     256 │        0    │ 109.05 │ 109.52 │ 39.85 │  133.59 │
╘══════════════╧══════════╧═════════╧═════════════╧════════╧════════╧═══════╧═════════╛


Oltre al nome del file, la tabella riporta:

- altezza dell'immagine, in pixel
- larghezza dell'immagine, in pixel
- se l'immagine è in scala di grigio, la colonna *grayscale* indica la media dei valori dell'unico livello di colore
- se l'immagine è a colori, le altre colonne indicano la media dei valori di ogni livello di colore.

---